#дз 3

сначала мы скопируем код с семинара, который по файлу json строит модельки


In [1]:
!pip install nltk
!pip install spacy
!pip  install pyLDAvis

     |████████████████████████████████| 1.6MB 7.2MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=19714ce2c00239eab76a709600aa4dafc4fbc2f3ebec7e26f050e60fc038c6ed
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [2]:
import nltk; nltk.download('stopwords')


!python3 -m spacy download en

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [4]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [52]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')

In [53]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

In [54]:
#tokenize

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [55]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [56]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [57]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [58]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

В этом месте мы закончили копировать семинар и начинаем программировать самостоятельно. Попробуем запустить маллет:

In [82]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = 'path/to/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)


# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


CalledProcessError: ignored

Не получается из-за CalledProcessError, та же проблема что у всех, значит не судьба.

Функция mod_coh считает coherence модели в зависимости от числа топиков.

In [59]:
def mod_coh(i):
  lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=i, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
  coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  return(coherence_lda)

Функция find_numtop находит оптимальное число топиков, перебирая модели с разным числом топиков и сравнивая их с помощью mod_coh.


In [60]:
def find_numtop():
  coh = 0
  numtop = 0
  for i in range(10, 50, 10):
    a = mod_coh(i)
    if a > coh:
      coh = a
      numtop = i
  return(numtop)

Запустим...........

In [61]:
numtop = find_numtop()

In [62]:
print(numtop)

10


Итак, мы знаем оптимальное число топиков, сделаем модель и запишем в переменную топики и соответствующие им слова.

In [63]:
topics = lda_model.show_topics(formatted=False, num_words= numtop)

Теперь создадим красивый словарик для соответствия топиков и слов.

In [64]:
twords = {}
for topic in topics:
  twords[topic[0]]=[]
  for tword in topic[1]:
    twords[topic[0]].append(tword[0])
print(twords)

{0: ['team', 'year', 'game', 'play', 'win', 'player', 'run', 'last', 'good', 'hit'], 1: ['go', 'time', 'day', 'come', 'take', 'back', 'get', 'say', 'see', 'first'], 2: ['space', 'cost', 'year', 'high', 'research', 'low', 'item', 'also', 'test', 'large'], 3: ['car', 'drive', 'bike', 'power', 'wire', 'slave', 'reality', 'speed', 'engine', 'light'], 4: ['ax', 'max', 'di_di', 'tumor', 'homosexual', 'gay', 'taste', 'liar', 'marry', 'homosexuality'], 5: ['government', 'people', 'gun', 'state', 'kill', 'year', 'public', 'attack', 'patient', 'country'], 6: ['line', 'thank', 'host', 'use', 'problem', 'get', 'work', 'need', 'system', 'window'], 7: ['would', 'write', 'article', 'line', 'know', 'think', 'make', 'be', 'may', 'say'], 8: ['evidence', 'believe', 'reason', 'faith', 'say', 'sense', 'claim', 'exist', 'law', 'physical'], 9: ['key', 'program', 'file', 'information', 'use', 'system', 'available', 'source', 'image', 'include']}


Сделаем словарь, который каждому тексту сопоставляет самый подходящий широкий топик и его суммарный вес.

In [65]:
from collections import Counter

In [66]:
text_top = {}
textid = -1
for text in texts:
  textid += 1
  cntr = Counter()
  for word in text:
    for t in topics:
      for tw in t[1]:
        if tw[0] == word:
          cntr[t[0]] += tw[1]
  text_top[textid] = cntr.most_common(1)
print(text_top)

{0: [(3, 0.17340180464088917)], 1: [(6, 0.06769515294581652)], 2: [(7, 0.14003772847354412)], 3: [(7, 0.13913768902420998)], 4: [(7, 0.13853340223431587)], 5: [(7, 0.38625595811754465)], 6: [(6, 0.08662551827728748)], 7: [(7, 0.16442767530679703)], 8: [(7, 0.0505372378975153)], 9: [(7, 0.1931290840730071)], 10: [(6, 0.10018170904368162)], 11: [(7, 0.4333155369386077)], 12: [(6, 0.036276303231716156)], 13: [(7, 0.19000880606472492)], 14: [(6, 0.08072984591126442)], 15: [(7, 0.1700465464964509)], 16: [(7, 0.19499597512185574)], 17: [(0, 0.7552270404994488)], 18: [(7, 0.14078723825514317)], 19: [(6, 0.10509959608316422)], 20: [(7, 0.07249696180224419)], 21: [(7, 0.11167267896234989)], 22: [(7, 0.1323595605790615)], 23: [(6, 0.12050658464431763)], 24: [(6, 0.10724369063973427)], 25: [(7, 0.1498113926500082)], 26: [(7, 0.16314878221601248)], 27: [(6, 0.05091208126395941)], 28: [(7, 0.0812458610162139)], 29: [(3, 0.07474146131426096)], 30: [(7, 0.24048450216650963)], 31: [(7, 0.0769927632063

Теперь нам надо вывернуть словарь наизнанку: каждому широкому топику сопоставить список текстов.

In [67]:
top_texts = {}
for i in range(len(topics)):
  top_texts[i] = []
  for textid, topic in text_top.items():
    if topic != []:
      if topic[0][0] == i:
        top_texts[i].append(textid)
print(top_texts)

{0: [17, 43, 49, 208, 256, 258, 266, 334, 345, 372, 388, 471, 472, 473, 479, 482, 494, 499, 502, 516, 525, 586, 664, 665, 666, 683, 724, 750, 753, 777, 780, 782, 863, 882, 897, 907, 920, 923, 957, 963, 968, 991, 1032, 1036, 1130, 1155, 1197, 1205, 1211, 1215, 1222, 1313, 1326, 1365, 1374, 1389, 1403, 1439, 1485, 1504, 1523, 1542, 1553, 1570, 1573, 1589, 1606, 1630, 1640, 1645, 1651, 1718, 1747, 1754, 1757, 1765, 1827, 1847, 1868, 1880, 1887, 1916, 1919, 1920, 1971, 1996, 2018, 2032, 2048, 2083, 2084, 2173, 2207, 2247, 2251, 2260, 2278, 2280, 2287, 2290, 2316, 2397, 2410, 2458, 2471, 2628, 2645, 2668, 2719, 2739, 2753, 2777, 2829, 2833, 2839, 2852, 2898, 2905, 2936, 2942, 3042, 3045, 3100, 3104, 3115, 3124, 3150, 3169, 3242, 3298, 3313, 3318, 3366, 3377, 3393, 3407, 3409, 3441, 3446, 3458, 3484, 3563, 3576, 3603, 3604, 3625, 3644, 3664, 3697, 3760, 3850, 3870, 3892, 3909, 3923, 3944, 3995, 4011, 4012, 4023, 4032, 4061, 4064, 4078, 4113, 4146, 4152, 4156, 4162, 4189, 4193, 4200, 4212, 42

Теперь приступаем к подсчету tf_idf через sklearn.

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [77]:
vectorizer = TfidfVectorizer()

Создадим словарь top_realtexts: в нем каждому широкому топику сопоставляется список лемматизированных текстов в виде строк.

In [70]:
top_realtexts = {}
for i, j in top_texts.items():
  top_realtexts[i] = [' '.join(texts[k]) for k in j]

Эта функция находит top_n лучших слов для строчки в табличке (я позаиствовала её с просторов гитхаба).

In [71]:
def get_top_tf_idf_words(tfidf_vector, feature_names, top_n):
    sorted_nzs = np.argsort(tfidf_vector.data)[:-(top_n+1):-1]
    return feature_names[tfidf_vector.indices[sorted_nzs]]

И всё, мы почти закончили! Пройдёмся по всему словарю top_realtexts, с помощью предыдущей функции вычленим из каждого текста 5 главных слов и запишем всё в csv-файл.

In [81]:
with open('final.csv', 'w', encoding='utf-8') as f:
  for j in range(len(top_realtexts)):
    vectors = vectorizer.fit_transform(top_realtexts[j])
    feature_names = np.array(vectorizer.get_feature_names())
    for i, text in enumerate(top_realtexts[j]):
      vector = vectors[i, :]
      words = get_top_tf_idf_words(vector, feature_names, 5)
      f.write(text + ',' + str(j) + ',' + ' '.join(words) + '\n')

Всё, csv файл готов, спасибо за внимание...